<a href="https://www.kaggle.com/code/sanjanchaudhari/rfm-case-study-project?scriptVersionId=137950988" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from datetime import datetime
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/recency-frequency-monetary-value/rfm_data.csv


In [2]:
rfm=pd.read_csv(r"/kaggle/input/recency-frequency-monetary-value/rfm_data.csv")
rfm

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location
0,8814,2023-04-11,943.31,Product C,890075,Tokyo
1,2188,2023-04-11,463.70,Product A,176819,London
2,4608,2023-04-11,80.28,Product A,340062,New York
3,2559,2023-04-11,221.29,Product A,239145,London
4,9482,2023-04-11,739.56,Product A,194545,Paris
...,...,...,...,...,...,...
995,2970,2023-06-10,759.62,Product B,275284,London
996,6669,2023-06-10,941.50,Product C,987025,New York
997,8836,2023-06-10,545.36,Product C,512842,London
998,1440,2023-06-10,729.94,Product B,559753,Paris


In [3]:
rfm.head(6)

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location
0,8814,2023-04-11,943.31,Product C,890075,Tokyo
1,2188,2023-04-11,463.70,Product A,176819,London
2,4608,2023-04-11,80.28,Product A,340062,New York
3,2559,2023-04-11,221.29,Product A,239145,London
4,9482,2023-04-11,739.56,Product A,194545,Paris
5,8483,2023-04-11,375.23,Product C,691194,Paris


# Step 1: Data Preparation

In [4]:
rfm = rfm.drop_duplicates()

In [5]:
rfm.head(5)

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location
0,8814,2023-04-11,943.31,Product C,890075,Tokyo
1,2188,2023-04-11,463.70,Product A,176819,London
2,4608,2023-04-11,80.28,Product A,340062,New York
3,2559,2023-04-11,221.29,Product A,239145,London
4,9482,2023-04-11,739.56,Product A,194545,Paris


In [6]:
columns_to_drop = ["ProductInformation", "Location"]
rfm = rfm.drop(columns=columns_to_drop)

In [7]:
rfm["PurchaseDate"] = pd.to_datetime(rfm["PurchaseDate"])
rfm["PurchaseDate"]

0     2023-04-11
1     2023-04-11
2     2023-04-11
3     2023-04-11
4     2023-04-11
         ...    
995   2023-06-10
996   2023-06-10
997   2023-06-10
998   2023-06-10
999   2023-06-10
Name: PurchaseDate, Length: 1000, dtype: datetime64[ns]

In [8]:
print(rfm.head())

   CustomerID PurchaseDate  TransactionAmount  OrderID
0        8814   2023-04-11             943.31   890075
1        2188   2023-04-11             463.70   176819
2        4608   2023-04-11              80.28   340062
3        2559   2023-04-11             221.29   239145
4        9482   2023-04-11             739.56   194545


# Step 2: Calculate RFM Metrics

In [9]:
# Calculate Recency (R)
current_date = datetime.now()  # Get the current date
rfm['PurchaseDate'] = pd.to_datetime(rfm['PurchaseDate'])  # Convert 'PurchaseDate' column to datetime
rfm['Recency'] = (current_date - rfm['PurchaseDate']).dt.days


print(rfm)

     CustomerID PurchaseDate  TransactionAmount  OrderID  Recency
0          8814   2023-04-11             943.31   890075      106
1          2188   2023-04-11             463.70   176819      106
2          4608   2023-04-11              80.28   340062      106
3          2559   2023-04-11             221.29   239145      106
4          9482   2023-04-11             739.56   194545      106
..          ...          ...                ...      ...      ...
995        2970   2023-06-10             759.62   275284       46
996        6669   2023-06-10             941.50   987025       46
997        8836   2023-06-10             545.36   512842       46
998        1440   2023-06-10             729.94   559753       46
999        4759   2023-06-10             804.28   467544       46

[1000 rows x 5 columns]


In [10]:
# Calculate Frequency (F)
rfm_frequency = rfm.groupby('CustomerID')['PurchaseDate'].count().reset_index()
rfm_frequency.columns = ['CustomerID', 'Frequency']
rfm = rfm.merge(rfm_frequency, on='CustomerID', how='left')

In [11]:
print(rfm)

     CustomerID PurchaseDate  TransactionAmount  OrderID  Recency  Frequency
0          8814   2023-04-11             943.31   890075      106          1
1          2188   2023-04-11             463.70   176819      106          1
2          4608   2023-04-11              80.28   340062      106          1
3          2559   2023-04-11             221.29   239145      106          1
4          9482   2023-04-11             739.56   194545      106          1
..          ...          ...                ...      ...      ...        ...
995        2970   2023-06-10             759.62   275284       46          1
996        6669   2023-06-10             941.50   987025       46          1
997        8836   2023-06-10             545.36   512842       46          1
998        1440   2023-06-10             729.94   559753       46          1
999        4759   2023-06-10             804.28   467544       46          1

[1000 rows x 6 columns]


In [12]:
# Calculate Monetary Value (M)
rfm_monetary = rfm.groupby('CustomerID')['TransactionAmount'].sum().reset_index()
rfm_monetary.columns = ['CustomerID', 'MonetaryValue']
rfm = rfm.merge(rfm_monetary, on='CustomerID', how='left')
print(rfm)

     CustomerID PurchaseDate  TransactionAmount  OrderID  Recency  Frequency  \
0          8814   2023-04-11             943.31   890075      106          1   
1          2188   2023-04-11             463.70   176819      106          1   
2          4608   2023-04-11              80.28   340062      106          1   
3          2559   2023-04-11             221.29   239145      106          1   
4          9482   2023-04-11             739.56   194545      106          1   
..          ...          ...                ...      ...      ...        ...   
995        2970   2023-06-10             759.62   275284       46          1   
996        6669   2023-06-10             941.50   987025       46          1   
997        8836   2023-06-10             545.36   512842       46          1   
998        1440   2023-06-10             729.94   559753       46          1   
999        4759   2023-06-10             804.28   467544       46          1   

     MonetaryValue  
0           943.31

# Step 3: Assign RFM Scores

In [13]:
rfm['R_score'] = pd.qcut(rfm['Recency'], q=5, labels=[5, 4, 3, 2, 1])
rfm['F_score'] = pd.qcut(rfm['Frequency'], q=5, labels=False, duplicates='drop')
rfm['M_score'] = pd.qcut(rfm['MonetaryValue'], q=5, labels=[1, 2, 3, 4, 5])
print(rfm['R_score'])
print(rfm['F_score'])
print(rfm['M_score'])

0      1
1      1
2      1
3      1
4      1
      ..
995    5
996    5
997    5
998    5
999    5
Name: R_score, Length: 1000, dtype: category
Categories (5, int64): [5 < 4 < 3 < 2 < 1]
0      0
1      0
2      0
3      0
4      0
      ..
995    0
996    0
997    0
998    0
999    0
Name: F_score, Length: 1000, dtype: int64
0      5
1      3
2      1
3      1
4      4
      ..
995    4
996    5
997    3
998    4
999    4
Name: M_score, Length: 1000, dtype: category
Categories (5, int64): [1 < 2 < 3 < 4 < 5]


# step 4: Calculate RFM Score

In [14]:
# Concatenate R_score, F_score, and M_score to create RFM_score
rfm['RFM_score'] = rfm['R_score'].astype(str) + rfm['F_score'].astype(str) + rfm['M_score'].astype(str)
print(rfm['RFM_score'])

0      105
1      103
2      101
3      101
4      104
      ... 
995    504
996    505
997    503
998    504
999    504
Name: RFM_score, Length: 1000, dtype: object


# Step 6: Analyze Segments

In [15]:
# Calculate the average RFM scores for each segment
segment_analysis = rfm.groupby('RFM_score').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'MonetaryValue': 'mean',
})
print(segment_analysis)

              Recency  Frequency  MonetaryValue
RFM_score                                      
101        100.627907   1.000000     117.040465
102        100.783784   1.027027     347.094865
103         99.632653   1.061224     566.751837
104         99.394737   1.105263     769.234474
105        100.666667   1.454545    1089.948182
201         87.857143   1.028571     123.719429
202         88.868421   1.026316     345.010000
203         88.928571   1.071429     567.468095
204         88.575000   1.050000     754.867500
205         88.363636   1.568182    1128.176591
301         79.171429   1.028571     125.483429
302         78.487805   1.048780     346.087317
303         77.676471   1.058824     558.900294
304         78.125000   1.100000     769.326000
305         78.200000   1.400000    1089.050889
401         63.767442   1.023256     129.488605
402         64.560976   1.000000     321.469512
403         64.750000   1.050000     568.269000
404         64.658537   1.073171     761

In [16]:
# Identify high-value customers
high_value_customers = segment_analysis[segment_analysis['MonetaryValue'] == segment_analysis['MonetaryValue'].max()]
print(high_value_customers)

             Recency  Frequency  MonetaryValue
RFM_score                                     
205        88.363636   1.568182    1128.176591


In [17]:
# Identify at-risk customers
at_risk_customers = segment_analysis[segment_analysis['Recency'] == segment_analysis['Recency'].min()]
print(at_risk_customers)

             Recency  Frequency  MonetaryValue
RFM_score                                     
501        52.204545   1.022727     117.601591


In [18]:
# Identify potential opportunities
potential_opportunities = segment_analysis[(segment_analysis['Frequency']
== segment_analysis['Frequency'].min()) & (segment_analysis['MonetaryValue'] == segment_analysis['MonetaryValue'].max())]
print(potential_opportunities)

Empty DataFrame
Columns: [Recency, Frequency, MonetaryValue]
Index: []
